# Using an API #

APIs are the best way to automatically download and parse data from the internet. For one, they are an explicitely approved interface whose primary purpose is to provide you with access to the data. They also provide the data in a format the is (usually) much easier to parse than HTML.

There are two main types of API interfaces: REST and SOAP. REST interfaces provide JSON data and SOAP interfaces usually provide XML. I think you'll find that REST is more common and probably a little easier to work with. But there are advantages to XML that we won't go into here.

In this notebook, we will use a Football API to access statistics from the Premier League in England (Football == Soccer). If you want to work through this notebook on your own, you will need to create your own account.

## References ##
* [SOAP vs REST vs JSON - a 2021 Comparison by RAYGUN.com](https://raygun.com/blog/soap-vs-rest-vs-json/)
* [Premier League Football Website](https://www.premierleague.com)
* [API Documentation at API-Football](https://www.api-football.com/documentation-v3)
* [API Documentation at RapidAPI](https://rapidapi.com/api-sports/api/api-football/)
* [Reddit about WUnderground Alternatives](https://www.reddit.com/r/webdev/comments/8tjavu/now_that_the_free_wunderground_api_has_been/)
* [Best Free Alternatives to the WUnderground Weather API](https://medium.com/@imdipto/best-free-alternatives-to-the-wunderground-weather-api-21acb22450e6)

In [1]:
# FIRST, LOAD UP THIS PAGE IN A BROWSER AND SEE WHAT WE ARE TALKING ABOUT
# https://www.premierleague.com/stats/top/players/goals

import urllib.request

url = 'https://www.premierleague.com/stats/top/players/goals'

response = urllib.request.urlopen(url)
content = response.read().decode('UTF-8')

print(content[:500])

<!DOCTYPE html>
<html lang="en">
<head>
    <meta name="twitter:title" content="Premier League Player Stats - Goals"/>
<meta name="keywords" content="Premier League, Football, Soccer, Official"/>
<meta property="og:type" content="website"/>
<meta name="description" content="View Goals played by Premier League players for 2018/19 and previous seasons, on the official website of the Premier League."/>
<meta name="twitter:description" content="View Goals played by Premier League players for 2018/19


In [2]:
if "Ronaldo" in content:
    index = content.index("Ronaldo")
    print(content[index-100:index+100])
else:
    print("Not Found")

Not Found


In [4]:
if "Shearer" in content:
    index = content.index("Shearer")
    print(content[index-99:index+500])


    <td class="rank"><strong>1</strong></td>
    <td scopr="row">
      <a href="/players/89/Alan-Shearer/overview" class="playerName">
      <strong>Alan Shearer</strong></a>
    </td>
    <td class="hide-s">
      -
    </td>
    <td class="hide-s">


        <div class="info">
          <span class="flag GB-ENG"></span>
          <span class="playerCountry">England</span>
        </div>
    </td>
    <td class="mainStat text-centre">260</td>
    <td>&nbsp;</td>
  </tr>



  <tr class="">
    <td class="rank"><strong>2</strong></td>
    <td scopr="row">
      <a href="/players/2064/Wayne-R


## Using the Football API ##

I will show these live in class but the sample code online will not include my API key.

To stay within the free limit, I recommend saving the results of each query to disk and then working from the saved file.

In [8]:
# Tutorial Example from https://rapidapi.com/api-sports/api/api-football/

import requests

url = "https://api-football-v1.p.rapidapi.com/v3/leagues"

headers = {
    "X-RapidAPI-Host": "api-football-v1.p.rapidapi.com",
    #"X-RapidAPI-Key": "a74d273292mshca9f4447e81622ep169b0cjsn7af70307b7be"
}

response = requests.request("GET", url, headers=headers)

print(response.text)

{"get":"leagues","parameters":[],"errors":[],"results":910,"paging":{"current":1,"total":1},"response":[{"league":{"id":4,"name":"Euro Championship","type":"Cup","logo":"https:\/\/media.api-sports.io\/football\/leagues\/4.png"},"country":{"name":"World","code":null,"flag":null},"seasons":[{"year":2008,"start":"2008-06-07","end":"2008-06-29","current":false,"coverage":{"fixtures":{"events":true,"lineups":true,"statistics_fixtures":false,"statistics_players":false},"standings":false,"players":false,"top_scorers":false,"top_assists":false,"top_cards":false,"injuries":false,"predictions":true,"odds":false}},{"year":2012,"start":"2012-06-08","end":"2012-07-01","current":false,"coverage":{"fixtures":{"events":true,"lineups":true,"statistics_fixtures":false,"statistics_players":false},"standings":false,"players":false,"top_scorers":false,"top_assists":false,"top_cards":false,"injuries":false,"predictions":true,"odds":false}},{"year":2016,"start":"2016-06-10","end":"2016-07-10","current":false

In [9]:
data = response.json()

In [10]:
data.keys()

dict_keys(['get', 'parameters', 'errors', 'results', 'paging', 'response'])

In [11]:
data['get']

'leagues'

In [12]:
data['parameters']

[]

In [13]:
data['errors']

[]

In [14]:
data['paging']

{'current': 1, 'total': 1}

In [15]:
data['results']

910

In [16]:
for entry in data['response']:
    print(entry.keys())
    print(entry['league'].keys())
    print(entry['country'].keys())
    for season in entry['seasons']:
        print(season.keys())
        break
    break

dict_keys(['league', 'country', 'seasons'])
dict_keys(['id', 'name', 'type', 'logo'])
dict_keys(['name', 'code', 'flag'])
dict_keys(['year', 'start', 'end', 'current', 'coverage'])


## What Data Can I Query? ## 

You'll have to read the API documentation to understand what queries are possible and how you perform them. A typical example will look like this:

```
url = "https://api-football-v1.p.rapidapi.com/v3/players/topscorers"
querystring = {"league":"39","season":"2021"}
```

In the examples below, I have consulted the API and also performed some data exploration. I did not know any of these queries when I first started.

## Helper Functions ##

Let's make some helper functions to download various queries and save the results to disk. We want to save the information since we are on a limited request count (100/day). We'll save the JSON to disk and then load the saved version in order to work with it.

In [18]:
import json
import requests

def save_json_data(filename, data):
    with open(filename, 'w') as fout:
        json_string_data = json.dumps(data)
        fout.write(json_string_data)
        
def load_json_data(filename):
    with open(filename) as fin:
        json_data = json.load(fin)
        return json_data

    
# References:
#    https://rapidapi.com/api-sports/api/api-football/
#    https://www.api-football.com/documentation-v3

def download_json_data(filename, url, querystring):
        
    headers = {
        "X-RapidAPI-Host": "api-football-v1.p.rapidapi.com",
        #"X-RapidAPI-Key": "a74d273292mshca9f4447e81622ep169b0cjsn7af70307b7be"
    }
    response = requests.request("GET", url, headers=headers, params=querystring)

    json_data = response.json()
    save_json_data(filename, json_data)
    
    pages_left = json_data['paging']['total'] - json_data['paging']['current']
    result = { 'get':json_data['get'], 'parameters':json_data['parameters'], 
               'errors':json_data['errors'], 'results':json_data['results'],
               'pages_remaining':pages_left }
    return result


In [19]:
# Enumerate all of the leagues
url = "https://api-football-v1.p.rapidapi.com/v3/leagues"
querystring = None
filename = 'leagues.json'

download_json_data(filename, url, querystring)

{'get': 'leagues',
 'parameters': [],
 'errors': [],
 'results': 910,
 'pages_remaining': 0}

In [20]:
# Top Scorers in a certain league by season
url = "https://api-football-v1.p.rapidapi.com/v3/players/topscorers"
querystring = {"league":"39","season":"2021"}
filename = 'top_scorers_2021.json'

download_json_data(filename, url, querystring)

{'get': 'players/topscorers',
 'parameters': {'league': '39', 'season': '2021'},
 'errors': [],
 'results': 20,
 'pages_remaining': 1}

In [21]:
# Teams in a certain league
url = "https://api-football-v1.p.rapidapi.com/v3/teams"
querystring = {"league":"39","season":"2021"}
filename = 'premier_teams.json'

download_json_data(filename, url, querystring)

{'get': 'teams',
 'parameters': {'league': '39', 'season': '2021'},
 'errors': [],
 'results': 20,
 'pages_remaining': 0}

In [22]:
url = 'https://api-football-v1.p.rapidapi.com/v3/players/squads'
querystring = {"team":"63"}
filename = 'leeds_players.json'

download_json_data(filename, url, querystring)

{'get': 'players/squads',
 'parameters': {'team': '63'},
 'errors': [],
 'results': 1,
 'pages_remaining': 0}

In [23]:
url = 'https://api-football-v1.p.rapidapi.com/v3/players'
querystring = {"id":"19134", "season":"2021"}
filename = 'p_bamford_attacker_leads.json'

download_json_data('p_bamford_attacker_leads.json', url, querystring)

{'get': 'players',
 'parameters': {'id': '19134', 'season': '2021'},
 'errors': [],
 'results': 1,
 'pages_remaining': 0}

## Parsing the API JSON Data ##

Let's explore some of the JSON data and convert into CSV data will be easier for pandas to store. A few items worth mentioning:
* In the examples below, you'll notice that I converted to a CSV format but didn't go to the trouble of actually saving a CSV file. You'll want to actually save CSV files.
* I'm working off of the saved JSON data in order to save API queries.
* For the Football API, even if I go over my 100 query limit, each subsequent query is only $0.005. But if each of you were to use my API Key and to make a mistake in your program that caused an unfortunate and long loop, my credit card might rack up quite the bill.

In [24]:
json_data = load_json_data('leagues.json')
print(json_data)

{'get': 'leagues', 'parameters': [], 'errors': [], 'results': 910, 'paging': {'current': 1, 'total': 1}, 'response': [{'league': {'id': 4, 'name': 'Euro Championship', 'type': 'Cup', 'logo': 'https://media.api-sports.io/football/leagues/4.png'}, 'country': {'name': 'World', 'code': None, 'flag': None}, 'seasons': [{'year': 2008, 'start': '2008-06-07', 'end': '2008-06-29', 'current': False, 'coverage': {'fixtures': {'events': True, 'lineups': True, 'statistics_fixtures': False, 'statistics_players': False}, 'standings': False, 'players': False, 'top_scorers': False, 'top_assists': False, 'top_cards': False, 'injuries': False, 'predictions': True, 'odds': False}}, {'year': 2012, 'start': '2012-06-08', 'end': '2012-07-01', 'current': False, 'coverage': {'fixtures': {'events': True, 'lineups': True, 'statistics_fixtures': False, 'statistics_players': False}, 'standings': False, 'players': False, 'top_scorers': False, 'top_assists': False, 'top_cards': False, 'injuries': False, 'predictions

In [25]:
print(json_data['response'][0])

{'league': {'id': 4, 'name': 'Euro Championship', 'type': 'Cup', 'logo': 'https://media.api-sports.io/football/leagues/4.png'}, 'country': {'name': 'World', 'code': None, 'flag': None}, 'seasons': [{'year': 2008, 'start': '2008-06-07', 'end': '2008-06-29', 'current': False, 'coverage': {'fixtures': {'events': True, 'lineups': True, 'statistics_fixtures': False, 'statistics_players': False}, 'standings': False, 'players': False, 'top_scorers': False, 'top_assists': False, 'top_cards': False, 'injuries': False, 'predictions': True, 'odds': False}}, {'year': 2012, 'start': '2012-06-08', 'end': '2012-07-01', 'current': False, 'coverage': {'fixtures': {'events': True, 'lineups': True, 'statistics_fixtures': False, 'statistics_players': False}, 'standings': False, 'players': False, 'top_scorers': False, 'top_assists': False, 'top_cards': False, 'injuries': False, 'predictions': True, 'odds': False}}, {'year': 2016, 'start': '2016-06-10', 'end': '2016-07-10', 'current': False, 'coverage': {'f

In [26]:
for entry in json_data['response']:
    print(entry['league']['id'], entry['league']['name'], entry['country']['name'])

4 Euro Championship World
21 Confederations Cup World
61 Ligue 1 France
144 Jupiler Pro League Belgium
71 Serie A Brazil
39 Premier League England
78 Bundesliga 1 Germany
135 Serie A Italy
88 Eredivisie Netherlands
94 Primeira Liga Portugal
140 La Liga Spain
179 Premiership Scotland
180 Championship Scotland
1 World Cup World
803 Asian Games World
804 Caribbean Cup World
62 Ligue 2 France
2 UEFA Champions League World
311 1st Division Albania
310 Superliga Albania
186 Ligue 1 Algeria
187 Ligue 2 Algeria
586 West Bank Premier League Palestine
588 National League Myanmar
591 Pro League Trinidad-And-Tobago
335 Cup Ukraine
336 Druha Liga - Group A Ukraine
334 Persha Liga Ukraine
333 Premier League Ukraine
301 Arabian Gulf League United-Arab-Emirates
303 Division 1 United-Arab-Emirates
302 League Cup United-Arab-Emirates
269 Segunda Division Uruguay
202 Ligue Professionnelle 1 Tunisia
203 Super Lig Turkey
204 TFF 1. Lig Turkey
205 TFF 2. Lig Turkey
206 Cup Turkey
552 3. Lig - Group 1 Turkey

### Enumerate All Leagues ###

In [27]:
def lookup_league_id(json_data, league_name='Premier League', league_country='England'):

    if json_data['get'] != 'leagues':
        print(f"Invalid JSON Data: expected 'leagues' but recieved '{json_data['get']}'")
    
    matches = []
    fuzzy_matches = []
    for entry in json_data['response']:
        if entry['league']['name'] == league_name and entry['country']['name'] == league_country:
            matches.append(entry['league']['id'])
        elif entry['league']['name'].startswith(league_name):
            fuzzy_matches.append(entry['league']['id'])
    
    if len(matches) > 0:
        return matches
    else:
        return fuzzy_matches
    
json_data = load_json_data('leagues.json')
print(lookup_league_id(json_data, 'Premier League', 'England'))
print(lookup_league_id(json_data, 'Bundesliga', 'Germany'))

[39]
[78, 79]


In [28]:
def convert_league_to_csv(json_data):
    
    if json_data['get'] != 'leagues':
        print(f"Invalid JSON Data: expected 'leagues' but recieved '{json_data['get']}'")
    
    rows = ['id, name, country, type, first_season, last_season, logo']
    for entry in json_data['response']:
        first_season = min([season['year'] for season in entry['seasons']])
        last_season = max([season['year'] for season in entry['seasons']])                            
        line = f"{entry['league']['id']}, {entry['league']['name']}, {entry['league']['type']}, " + \
               f"{first_season}, {last_season}, {entry['league']['logo']}"
        rows.append(line)
    
    return rows

json_data = load_json_data('leagues.json')
convert_league_to_csv(json_data)

['id, name, country, type, first_season, last_season, logo',
 '4, Euro Championship, Cup, 2008, 2020, https://media.api-sports.io/football/leagues/4.png',
 '21, Confederations Cup, Cup, 2009, 2017, https://media.api-sports.io/football/leagues/21.png',
 '61, Ligue 1, League, 2010, 2021, https://media.api-sports.io/football/leagues/61.png',
 '144, Jupiler Pro League, League, 2010, 2021, https://media.api-sports.io/football/leagues/144.png',
 '71, Serie A, League, 2010, 2022, https://media.api-sports.io/football/leagues/71.png',
 '39, Premier League, League, 2010, 2021, https://media.api-sports.io/football/leagues/39.png',
 '78, Bundesliga 1, League, 2010, 2021, https://media.api-sports.io/football/leagues/78.png',
 '135, Serie A, League, 2010, 2021, https://media.api-sports.io/football/leagues/135.png',
 '88, Eredivisie, League, 2010, 2021, https://media.api-sports.io/football/leagues/88.png',
 '94, Primeira Liga, League, 2010, 2021, https://media.api-sports.io/football/leagues/94.png',


### Find Top-20 Premier League Scorers for 2021 ###

In [29]:
json_data = load_json_data('top_scorers_2021.json')

print(json_data['response'][0].keys())
print()
print(json_data['response'][0]['player'])
print()
print(json_data['response'][0]['statistics'])
print()
print()

dict_keys(['player', 'statistics'])

{'id': 306, 'name': 'Mohamed Salah Ghaly', 'firstname': 'Mohamed', 'lastname': 'Salah Ghaly', 'age': 30, 'birth': {'date': '1992-06-15', 'place': 'Muḥāfaẓat al Gharbiyya', 'country': 'Egypt'}, 'nationality': 'Egypt', 'height': '175 cm', 'weight': '71 kg', 'injured': False, 'photo': 'https://media.api-sports.io/football/players/306.png'}

[{'team': {'id': 40, 'name': 'Liverpool', 'logo': 'https://media.api-sports.io/football/teams/40.png'}, 'league': {'id': 39, 'name': 'Premier League', 'country': 'England', 'logo': 'https://media.api-sports.io/football/leagues/39.png', 'flag': 'https://media.api-sports.io/flags/gb.svg', 'season': 2021}, 'games': {'appearences': 28, 'lineups': 26, 'minutes': 2331, 'number': None, 'position': 'Attacker', 'rating': '7.439285', 'captain': False}, 'substitutes': {'in': 2, 'out': 5, 'bench': 2}, 'shots': {'total': 89, 'on': 54}, 'goals': {'total': 20, 'conceded': 0, 'assists': 10, 'saves': None}, 'passes': {'total': 909, 

In [30]:
for scorer in json_data['response']:
    print(f"{scorer['player']['name']} ({scorer['player']['nationality']}) ({scorer['player']['id']}): " +
          f"{scorer['player']['height']} / {scorer['player']['weight']}")
    for stats in scorer['statistics']:
        print(f"  {stats['league']['season']} {stats['team']['name']}: Games: {stats['games']['appearences']} " +
              f"Minutes: {stats['games']['minutes']} " + 
              f"Shots On: {stats['shots']['on']}/{stats['shots']['total']} " +
              f"Goals: {stats['goals']['total']} Assists: {stats['goals']['assists']}")
    print()

Mohamed Salah Ghaly (Egypt) (306): 175 cm / 71 kg
  2021 Liverpool: Games: 28 Minutes: 2331 Shots On: 54/89 Goals: 20 Assists: 10

Heung-Min Son (Korea Republic) (186): 183 cm / 78 kg
  2021 Tottenham: Games: 27 Minutes: 2342 Shots On: 34/51 Goals: 14 Assists: 6

Diogo Jota (Portugal) (2678): 178 cm / 68 kg
  2021 Liverpool: Games: 27 Minutes: 1848 Shots On: 27/58 Goals: 14 Assists: 1

Sadio Mané (Senegal) (304): 175 cm / 69 kg
  2021 Liverpool: Games: 27 Minutes: 2252 Shots On: 31/53 Goals: 12 Assists: 1

Harry Kane (England) (184): 188 cm / 86 kg
  2021 Tottenham: Games: 29 Minutes: 2512 Shots On: 41/74 Goals: 12 Assists: 6

Cristiano Ronaldo (Portugal) (874): 187 cm / 83 kg
  2021 Manchester United: Games: 24 Minutes: 1919 Shots On: 33/62 Goals: 12 Assists: 3

I. Toney (England) (19974): 179 cm / 65 kg
  2021 Brentford: Games: 26 Minutes: 2278 Shots On: 26/47 Goals: 11 Assists: 3

Kevin De Bruyne (Belgium) (629): 181 cm / 68 kg
  2021 Manchester City: Games: 23 Minutes: 1615 Shots O

## Enumerate All Permier League Teams ##

In [31]:
def convert_teams_to_csv(json_data):
    
    if json_data['get'] != 'teams':
        print(f"Invalid JSON Data: expected 'teams' but recieved '{json_data['get']}'")
    
    rows = ['league, season, id, name, code, country, founded, national, stadium, city, surface, logo']
    for entry in json_data['response']:
        line = \
        f"{json_data['parameters']['league']}, {json_data['parameters']['season']}, " + \
        f"{entry['team']['id']}, {entry['team']['name']}, {entry['team']['code']}, " + \
        f"{entry['team']['country']}, {entry['team']['founded']}, {entry['team']['national']}, " + \
        f"{entry['venue']['name']}, {entry['venue']['city']}, {entry['venue']['surface']}, {entry['team']['logo']}"
        rows.append(line)
    
    return rows

json_data = load_json_data('premier_teams.json')
csv_rows = convert_teams_to_csv(json_data)
csv_rows

['league, season, id, name, code, country, founded, national, stadium, city, surface, logo',
 '39, 2021, 33, Manchester United, MUN, England, 1878, False, Old Trafford, Manchester, grass, https://media.api-sports.io/football/teams/33.png',
 "39, 2021, 34, Newcastle, NEW, England, 1892, False, St. James' Park, Newcastle upon Tyne, grass, https://media.api-sports.io/football/teams/34.png",
 '39, 2021, 38, Watford, WAT, England, 1881, False, Vicarage Road, Watford, grass, https://media.api-sports.io/football/teams/38.png',
 '39, 2021, 39, Wolves, WOL, England, 1877, False, Molineux Stadium, Wolverhampton, West Midlands, grass, https://media.api-sports.io/football/teams/39.png',
 '39, 2021, 40, Liverpool, LIV, England, 1892, False, Anfield, Liverpool, grass, https://media.api-sports.io/football/teams/40.png',
 "39, 2021, 41, Southampton, SOU, England, 1885, False, St. Mary's Stadium, Southampton, Hampshire, grass, https://media.api-sports.io/football/teams/41.png",
 '39, 2021, 42, Arsenal,

### All Players on Leads Squad ###

In [32]:
def convert_roster_to_csv(json_data):
    
    if json_data['get'] != 'players/squads':
        print(f"Invalid JSON Data: expected 'players/squads' but recieved '{json_data['get']}'")
    
    rows = ['team_id, team_name, id, name, age, number, position, photo']
    for entry in json_data['response']:
        for player in entry['players']:
            line = \
            f"{entry['team']['id']}, {entry['team']['name']}, " + \
            f"{player['id']}, {player['name']}, {player['age']}, " + \
            f"{player['number']}, {player['position']}, {player['photo']}"
            rows.append(line)
    
    return rows

json_data = load_json_data('leeds_players.json')
csv_rows = convert_roster_to_csv(json_data)
csv_rows

['team_id, team_name, id, name, age, number, position, photo',
 '63, Leeds, 20619, I. Meslier, 22, 1, Goalkeeper, https://media.api-sports.io/football/players/20619.png',
 '63, Leeds, 39105, K. Klaesson, 22, 13, Goalkeeper, https://media.api-sports.io/football/players/39105.png',
 '63, Leeds, 19116, L. Ayling, 31, 2, Defender, https://media.api-sports.io/football/players/19116.png',
 '63, Leeds, 1564, Junior Firpo, 26, 3, Defender, https://media.api-sports.io/football/players/1564.png',
 '63, Leeds, 26238, R. Koch, 26, 5, Defender, https://media.api-sports.io/football/players/26238.png',
 '63, Leeds, 19118, L. Cooper, 31, 6, Defender, https://media.api-sports.io/football/players/19118.png',
 '63, Leeds, 47302, Diego Llorente, 29, 14, Defender, https://media.api-sports.io/football/players/47302.png',
 '63, Leeds, 19126, S. Dallas, 31, 15, Defender, https://media.api-sports.io/football/players/19126.png',
 '63, Leeds, 64003, P. Struijk, 23, 21, Defender, https://media.api-sports.io/footb

### Statistics for Single Player ###

In [33]:
def convert_player_to_csv(json_data):
    
    if json_data['get'] != 'players':
        print(f"Invalid JSON Data: expected 'players' but recieved '{json_data['get']}'")
    
    header = 'id, name, age, height, weight, photo, injured, team, league, season, games, position, minutes, ' + \
    'rating, captain, shots_taken, shots_on, goals, assists, passes, accuracy, tackles, blocks, interceptions, ' + \
    'duels, duels_won, dribble_attempts, dribble_success, fouls_drawn, fouls_committed, penalty_scored, penalty_missed'
    rows = [header]
    for entry in json_data['response']:
        for stats in entry['statistics']:
            line = \
            f"{entry['player']['id']}, {entry['player']['name']}, {entry['player']['age']}, {entry['player']['height']}, " + \
            f"{entry['player']['weight']}, {entry['player']['photo']}, {entry['player']['injured']}, " + \
            f"{stats['team']['name']}, {stats['league']['name']}, {stats['league']['season']}, " + \
            f"{stats['games']['appearences']}, {stats['games']['position']}, {stats['games']['minutes']}, " + \
            f"{stats['games']['rating']}, {stats['games']['captain']}, " + \
            f"{stats['shots']['total']}, {stats['shots']['on']}, " + \
            f"{stats['goals']['total']}, {stats['goals']['assists']}, " + \
            f"{stats['passes']['total']}, {stats['passes']['accuracy']}, " + \
            f"{stats['tackles']['total']}, {stats['tackles']['blocks']}, {stats['tackles']['interceptions']}, " + \
            f"{stats['duels']['total']}, {stats['duels']['won']}, " + \
            f"{stats['dribbles']['attempts']}, {stats['dribbles']['success']}, " + \
            f"{stats['fouls']['drawn']}, {stats['fouls']['committed']}, " + \
            f"{stats['penalty']['scored']}, {stats['penalty']['missed']}"

            rows.append(line)
    
    return rows

json_data = load_json_data('p_bamford_attacker_leads.json')
csv_rows = convert_player_to_csv(json_data)
csv_rows

['id, name, age, height, weight, photo, injured, team, league, season, games, position, minutes, rating, captain, shots_taken, shots_on, goals, assists, passes, accuracy, tackles, blocks, interceptions, duels, duels_won, dribble_attempts, dribble_success, fouls_drawn, fouls_committed, penalty_scored, penalty_missed',
 '19134, P. Bamford, 29, 185 cm, 71 kg, https://media.api-sports.io/football/players/19134.png, False, Leeds, Premier League, 2021, 9, Attacker, 560, 7.011111, False, 16, 5, 2, 2, 104, 9, 1, 2, 2, 43, 15, 7, 4, 5, 5, 0, 0',
 '19134, P. Bamford, 29, 185 cm, 71 kg, https://media.api-sports.io/football/players/19134.png, False, Leeds, League Cup, 2021, 1, Attacker, 21, 7.000000, False, 3, 2, 0, 1, 7, 7, None, None, 1, 3, 1, 3, 1, None, None, 0, 0',
 '19134, P. Bamford, 29, 185 cm, 71 kg, https://media.api-sports.io/football/players/19134.png, False, Leeds, FA Cup, 2021, 0, Attacker, 0, None, False, None, None, 0, None, None, None, None, None, None, None, None, None, None, Non